In [ ]:
import disco
import numpy as np
import h5py
from matplotlib import pyplot as plt
from astropy import units as u
from astropy import constants
from scipy.constants import elementary_charge
import ai.cs
from mpl_toolkits.mplot3d import Axes3D
import time
import pandas as pd

import cupy 
cupy.cuda.runtime.setDevice(3)

# Load Regridded File

In [ ]:
hdf = h5py.File('regrid.h5')
regrid_data = {}

for key in hdf.keys():
    regrid_data[key] = hdf[key][:]
hdf.close()

regrid_data['Bx'] *= u.nT
regrid_data['By'] *= u.nT
regrid_data['Bz'] *= u.nT
regrid_data['Ex'] *= u.mV/u.m
regrid_data['Ey'] *= u.mV/u.m
regrid_data['Ez'] *= u.mV/u.m
regrid_data['n'] *= u.cm**(-3)
regrid_data['T'] *= u.eV
regrid_data['xaxis'] *= u.R_earth
regrid_data['yaxis'] *= u.R_earth
regrid_data['zaxis'] *= u.R_earth

# Make Axes Instance

In [ ]:
taxis = np.array([-1, 1]) * u.day

axes = disco.Axes(regrid_data['xaxis'],
                  regrid_data['yaxis'],
                  regrid_data['zaxis'],
                  taxis,
                  r_inner=2.5 * u.R_earth)
axes

In [ ]:
regrid_data['Bx'].shape

# Make FieldModel Instance

In [ ]:
mass = constants.m_p
charge = elementary_charge * u.coulomb

Bx = np.array([regrid_data['Bx'].T]*taxis.size).T * regrid_data['Bx'].unit
By = np.array([regrid_data['By'].T]*taxis.size).T * regrid_data['By'].unit
Bz = np.array([regrid_data['Bz'].T]*taxis.size).T * regrid_data['Bz'].unit
Ex = np.array([regrid_data['Ex'].T]*taxis.size).T * regrid_data['Ex'].unit
Ey = np.array([regrid_data['Ey'].T]*taxis.size).T * regrid_data['Ey'].unit
Ez = np.array([regrid_data['Ez'].T]*taxis.size).T * regrid_data['Ez'].unit

m = (Bx.value==0) & (By.value==0) & (Bz.value==0) & (Ex.value==0) & (Ey.value==0) & (Ez.value==0)
print('Cells to NaN:', m.sum() / m.size)
Bx[m] = np.nan
By[m] = np.nan
Bz[m] = np.nan
Ex[m] = np.nan
Ey[m] = np.nan
Ez[m] = np.nan

field_model = disco.FieldModel(Bx, By, Bz, Ex, Ey, Ez, mass, charge, axes)
field_model

# Make ParticleState

In [ ]:
particle_height = 3 * u.R_earth

particle_long_axis = np.deg2rad(np.arange(0, 360, .1)) # magnetic longitude
particle_invlat_axis = np.deg2rad(np.arange(65, 85, .1)) # invariant magnetic latitude
particle_lat_axis = np.arcsin(np.sqrt((u.R_earth / particle_height) * np.sin(particle_invlat_axis)**2))

particle_lat, particle_long = np.meshgrid(particle_long_axis, particle_lat_axis, indexing='ij')

pos_x, pos_y, pos_z = ai.cs.sp2cart(particle_height, particle_long, particle_lat, )

In [ ]:

ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')

uu = np.linspace(0, 2 * np.pi, 100)
vv = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(uu), np.sin(vv))
y = np.outer(np.sin(uu), np.sin(vv))
z = np.outer(np.ones(np.size(uu)), np.cos(vv))
ax.plot_surface(x, y, z)

ax.plot(pos_x, pos_y, pos_z, '.', zorder=100)
ax.set_xlim(-16, 16)
ax.set_ylim(-16, 16)
ax.set_zlim(-16, 16)
ax.set_xlabel('X (SM)')
ax.set_ylabel('Y (SM)')
ax.set_zlabel('Z (SM)')
ax.set_title('MARBLE Grid')

In [ ]:
energy_min = 5
energy_max = 20e3
energy_count = 30

energy_axis = 10**np.linspace(np.log10(energy_min), np.log10(energy_max), energy_count) * u.eV
energy_axis

In [ ]:
particle_lat, particle_long, particle_energy = np.meshgrid(particle_lat_axis, particle_long_axis, energy_axis, indexing='ij')
pos_x, pos_y, pos_z = ai.cs.sp2cart(particle_height, particle_long, particle_lat, )
pos_x *= u.R_earth
pos_y *= u.R_earth
pos_z *= u.R_earth
particle_vel = np.sqrt(2 * particle_energy / constants.m_p)
gamma = 1 / np.sqrt(1 - (particle_vel / constants.c) ** 2)
ppar = gamma * constants.m_p * particle_vel
magnetic_moment = np.zeros(ppar.shape) * u.MeV/u.nT

In [ ]:
particle_state = disco.ParticleState(pos_x.flatten(), pos_y.flatten(), pos_z.flatten(),
                                     ppar.flatten(), magnetic_moment.flatten(), constants.m_p, charge)

# Make a TraceConfig

In [ ]:
config = disco.TraceConfig(
    t_final=-5 * u.hr,
    h_initial=5 * u.ms,
    rtol=1e-2,
    integrate_backwards=True,
    iters_max=500,
)

# Perform Traces

In [ ]:
start_time = time.time()
hist = disco.trace_trajectory(config, particle_state, field_model)
print('took', time.time() - start_time, 's')

# Plotting

In [ ]:
plt.hist(-disco._undim_time(hist.t[-1, :].flatten()).to(u.min).value, bins=np.arange(60))
plt.yscale('log')
plt.xlabel('Time Backwards Integrated (minutes)')
plt.ylabel('Particles Finished (Bin Count)')
plt.title('Integration Time Required')
None

In [ ]:
xx = hist.x[-1, :]
yy = hist.y[-1, :]
zz = hist.z[-1, :]
tt = hist.t[-1, :]

threshold = 0.5

out_bd = (
    (np.abs(xx -  regrid_data['xaxis'].value[0]) < threshold) |
    (np.abs(xx -  regrid_data['xaxis'].value[-1]) < threshold) |
    
    (np.abs(yy -  regrid_data['yaxis'].value[0]) < threshold) |
    (np.abs(yy -  regrid_data['yaxis'].value[-1]) < threshold) |
    
    (np.abs(zz -  regrid_data['zaxis'].value[0]) < threshold) |
    (np.abs(zz -  regrid_data['zaxis'].value[-1]) < threshold) 
)

inner_bd = (np.abs(np.sqrt(xx**2 + yy**2 + zz**2) - field_model.axes.r_inner) < 0.1)

In [ ]:
out_bd.sum() + inner_bd.sum()

In [ ]:
plt.bar([1, 2], [inner_bd.sum(), out_bd.sum()])
plt.gca().set_xticks([1, 2])
plt.gca().set_xticklabels(['Lost to Inner Boundary', 'Lost to Outer Boundary'])
plt.title('Particle Trace Endpoint')


In [ ]:
b = inner_bd | out_bd
df = pd.DataFrame(dict(
    x=hist.x[-1, b],
    y=hist.y[-1, b], 
    z=hist.z[-1, b],
    c=(particle_energy.flatten()/1e3).flatten()[b]
))
df.head()

In [ ]:
from matplotlib.colors import LogNorm
df_small = df.sample(n=int(1e6)).sample(frac=1)

plt.figure()
ax = plt.gca()
ax.plot(df_small.x, df_small.y, ',')
ax.set_xlim(-16, 16)
ax.set_ylim(-16, 16)
ax.set_xlabel('X (SM)')
ax.set_ylabel('Y (SM)')
ax.set_title('Backwards Tracing Endpoints - Equitorial')

plt.figure()
ax = plt.gca()
ax.plot(df_small.x, df_small.z, ',')
ax.set_xlim(-16, 16)
ax.set_ylim(-16, 16)
ax.set_xlabel('X (SM)')
ax.set_ylabel('Z (SM)')
ax.set_title('Backwards Tracing Endpoints - Meridonal')

# Inspect Inner Boundary Termination Trajectories

In [ ]:

n_in = 500

particle_state_in = disco.ParticleState(pos_x.flatten()[inner_bd][:n_in],
                                        pos_y.flatten()[inner_bd][:n_in],
                                        pos_z.flatten()[inner_bd][:n_in],
                                         ppar.flatten()[inner_bd][:n_in],
                                        magnetic_moment.flatten()[inner_bd][:n_in],
                                        constants.m_p,
                                        charge)

In [ ]:
config_in = disco.TraceConfig(
    t_final=-5 * u.hr,
    h_initial=5 * u.ms,
    rtol=1e-2,
    integrate_backwards=True,
    iters_max=500,
    output_freq=1
)

In [ ]:
start_time = time.time()
hist_new = disco.trace_trajectory(config_in, particle_state_in, field_model)
print('took', time.time() - start_time, 's')

In [ ]:
hist_new_r = np.sqrt(hist_new.x**2 + hist_new.y**2 + hist_new.z**2)
plt.plot(np.diff(hist_new_r, axis=0))
None

In [ ]:
plt.plot(hist.W[-1, :])

In [ ]:
c = particle_energy.flatten()[inner_bd][:n_in].value
cmap = plt.get_cmap('viridis')

mask = (hist_new.z > .5).any(axis=0)
print(mask.shape)

plt.figure()
ax = plt.gca()
for i in range(n_in):
    if mask[i]:
        #color = cmap(np.log10(c[i]) / np.log10(c).max())
        ax.scatter(hist_new.x[:, i], hist_new.y[:, i], c=hist_new.t[:, i], s=0.2)
ax.plot(hist_new.x[0, :], hist_new.y[0, :],  'k.')
ax.set_xlim(0, 3)
ax.set_ylim(-3, 3)
ax.set_xlabel('X (SM)')
ax.set_ylabel('Y (SM)')
ax.set_title('Northern Hemisphere Inner Boundary Loss - Equitorial')

plt.figure()
ax = plt.gca()
ax.plot(hist_new.x[0, :], hist_new.z[0, :], 'k.')
for i in range(n_in):
    if mask[i]:
        color = cmap(np.log10(c[i]) / np.log10(c).max())
        ax.scatter(hist_new.x[:, i], hist_new.z[:, i], c=hist_new.t[:, i], s=0.2)
ax.plot(hist_new.x[0, :], hist_new.z[0, :], 'k.')
ax.set_xlim(1.5, 3)
ax.set_ylim(-2, 1.5)
ax.set_xlabel('X (SM)')
ax.set_ylabel('Z (SM)')
ax.set_title('Northern Hemisphere Inner Boundary Loss - Meridonal')